# 1. <a id='toc1_'></a>[NBA Season 2022-2023 Analysis](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- 1. [NBA Season 2022-2023 Analysis](#toc1_)    
- 2. [Importings](#toc2_)    
  - 2.1. [Libraries](#toc2_1_)    
  - 2.2. [Helper Function](#toc2_2_)    
  - 2.3. [Data loading](#toc2_3_)    
- 3. [Data exploration and problem comprehension](#toc3_)    
  - 3.1. [Examining the **Advanced Dataset**](#toc3_1_)    
    - 3.1.1. [Features from Advanced Dataset](#toc3_1_1_)    
    - 3.1.2. [What are we dealing with?](#toc3_1_2_)    
    - 3.1.3. [Renaming and droping empty columns](#toc3_1_3_)    
    - 3.1.4. [Checking for NAs](#toc3_1_4_)    
    - 3.1.5. [Do these players have multiple lines due to team exchanges?](#toc3_1_5_)    
    - 3.1.6. [Let's combine the rows with same players](#toc3_1_6_)    
      - 3.1.6.1. [Checking if the concatenation went right](#toc3_1_6_1_)    
    - 3.1.7. [First glance at the Advanced Dataset](#toc3_1_7_)    
    - 3.1.8. [Imputing values to the missing data](#toc3_1_8_)    
  - 3.2. [Examining **Per Game Dataset**](#toc3_2_)    
    - 3.2.1. [Features from Per Game Dataset](#toc3_2_1_)    
    - 3.2.2. [What are we dealing with?](#toc3_2_2_)    
    - 3.2.3. [Renaming the columns](#toc3_2_3_)    
    - 3.2.4. [Checking for NAs](#toc3_2_4_)    
    - 3.2.5. [Let's combine multiple player rows in one](#toc3_2_5_)    
      - 3.2.5.1. [Checking if the concatanation went as expected](#toc3_2_5_1_)    
      - 3.2.5.2. [Checking again for NAs](#toc3_2_5_2_)    
    - 3.2.6. [Filling out NAs](#toc3_2_6_)    
    - 3.2.7. [First glance at the Per Game Dataset](#toc3_2_7_)    
- 4. [Feature Engineering and Hypothesis Creation](#toc4_)    
- 5. [Data selection and filtering](#toc5_)    
- 6. [Exploratory Data Analysis](#toc6_)    
- 7. [Data Preparation](#toc7_)    
- 8. [Feature Selection through Boruta algorithm](#toc8_)    
- 9. [Model implementation](#toc9_)    
- 10. [Hyperparameter Fine-Tuning](#toc10_)    
- 11. [Model Error Estimation and Interpretation](#toc11_)    
- 12. [Model Deployment](#toc12_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 2. <a id='toc2_'></a>[Importings](#toc0_)

## 2.1. <a id='toc2_1_'></a>[Libraries](#toc0_)

In [153]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pickle

from ydata_profiling        import ProfileReport
from sklearn.impute         import SimpleImputer
from IPython.display        import Image
from IPython.core.display   import HTML

## 2.2. <a id='toc2_2_'></a>[Helper Function](#toc0_)

In [154]:
def jupyter_configs():
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [15, 8]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    pd.set_option('display.max_columns', None)
    
    sns.set()
    
    warnings.filterwarnings( 'ignore' )
    
jupyter_configs()

## 2.3. <a id='toc2_3_'></a>[Data loading](#toc0_)

In [3]:
advanced_df_raw = pd.read_csv('./data/data_advanced.csv')
pergame_df_raw = pd.read_csv('./data/data_pergame.csv')

# 3. <a id='toc3_'></a>[Data exploration and problem comprehension](#toc0_)
- Main goal/problem
- Sub-goals
- What will the finished product be?

## 3.1. <a id='toc3_1_'></a>[Examining the **Advanced Dataset**](#toc0_)

### 3.1.1. <a id='toc3_1_1_'></a>[Features from Advanced Dataset](#toc0_)


- Rk -- Rank

- Pos -- Position

- Age -- Player's age on February 1 of the season

- Tm -- Team

- G -- Games

- MP -- Minutes Played

- PER -- Player Efficiency Rating. A measure of per-minute production standardized such that the league average is 15.

- TS% -- True Shooting Percentage. A measure of shooting efficiency that takes into account 2-point field goals, 3-point field goals, and free throws.

- 3PAr -- 3-Point Attempt Rate. Percentage of FG Attempts from 3-Point Range

- FTr -- Free Throw Attempt Rate. Number of FT Attempts Per FG Attempt

- ORB% -- Offensive Rebound Percentage. An estimate of the percentage of available offensive rebounds a player grabbed while they were on the floor.

- DRB% -- Defensive Rebound Percentage. An estimate of the percentage of available defensive rebounds a player grabbed while they were on the floor.

- TRB% -- Total Rebound Percentage. An estimate of the percentage of available rebounds a player grabbed while they were on the floor.

- AST% -- Assist Percentage. An estimate of the percentage of teammate field goals a player assisted while they were on the floor.

- STL% -- Steal Percentage. An estimate of the percentage of opponent possessions that end with a steal by the player while they were on the floor.

- BLK% -- Block Percentage. An estimate of the percentage of opponent two-point field goal attempts blocked by the player while they were on the floor.

- TOV% -- Turnover Percentage. An estimate of turnovers committed per 100 plays.

- USG% -- Usage Percentage. An estimate of the percentage of team plays used by a player while they were on the floor.

- OWS -- Offensive Win Shares. An estimate of the number of wins contributed by a player due to offense.

- DWS -- Defensive Win Shares. An estimate of the number of wins contributed by a player due to defense.

- WS -- Win Shares. An estimate of the number of wins contributed by a player.

- WS/48 -- Win Shares Per 48 Minutes. An estimate of the number of wins contributed by a player per 48 minutes (league average is approximately .100)

- OBPM -- Offensive Box Plus/Minus. A box score estimate of the offensive points per 100 possessions a player contributed above a league-average player, translated to an average team.

- DBPM -- Defensive Box Plus/Minus. A box score estimate of the defensive points per 100 possessions a player contributed above a league-average player, translated to an average team.

- BPM -- Box Plus/Minus. A box score estimate of the points per 100 possessions a player contributed above a league-average player, translated to an average team.

- VORP -- Value over Replacement Player. A box score estimate of the points per 100 TEAM possessions that a player contributed above a replacement-level (-2.0) player, translated to an average team and prorated to an 82-game season. Multiply by 2.70 to convert to wins over replacement.

### 3.1.2. <a id='toc3_1_2_'></a>[What are we dealing with?](#toc0_)

In [4]:
advanced_df_raw.head()

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player-additional
0,1,Precious Achiuwa,C,23,TOR,55,1140,15.2,0.554,0.267,...,0.8,1.4,2.2,0.093,NaN,-1.4,-0.8,-2.3,-0.1,achiupr01
1,2,Steven Adams,C,29,MEM,42,1133,17.5,0.564,0.004,...,1.3,2.1,3.4,0.144,NaN,-0.3,0.9,0.6,0.7,adamsst01
2,3,Bam Adebayo,C,25,MIA,75,2598,20.1,0.592,0.011,...,3.6,3.8,7.4,0.137,NaN,0.8,0.8,1.5,2.3,adebaba01
3,4,Ochai Agbaji,SG,22,UTA,59,1209,9.5,0.561,0.591,...,0.9,0.4,1.3,0.053,NaN,-1.7,-1.4,-3.0,-0.3,agbajoc01
4,5,Santi Aldama,PF,22,MEM,77,1682,13.9,0.591,0.507,...,2.1,2.4,4.6,0.130,NaN,-0.3,0.8,0.5,1.1,aldamsa01


In [5]:
advanced_df_raw.shape

(679, 30)

### 3.1.3. <a id='toc3_1_3_'></a>[Renaming and droping empty columns](#toc0_)

In [6]:
droped_columns = ['Unnamed: 19', 'Unnamed: 24']
advanced_df_raw = advanced_df_raw.drop(droped_columns, axis = 1)

In [7]:
advanced_df_raw.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Player-additional'],
      dtype='object')

In [8]:
advanced_cols = ['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'OWS', 'DWS', 'WS', 'WS_48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Player_additional']

advanced_df_raw.columns = advanced_cols

In [9]:
advanced_df_raw.shape

(679, 28)

In [10]:
# There are 679 rows in the dataset. However only 539 singular players. It happens because some players changed teams during the season and appear in multiple lines.
# It may be a good solution to join these lines and stick only with the latest team in wich the player acts.

print( advanced_df_raw['Player_additional'].nunique(), 'out of', advanced_df_raw.shape[0])

539 out of 679


### 3.1.4. <a id='toc3_1_4_'></a>[Checking for NAs](#toc0_)
- Only three NAs in columns 'TS%', '3PAr' and 'FTr', and one at the column 'TOV%'. The same three rows have NAs to the first three features and Michael Foster Jr. has missing values to 'TOV%'. 
- Let's inspect it so we can figure out why they are empty and what to do with it.
- Columns 'Unnamed: 19' and 'Unnamed: 24' are completely empty and should be deleted.

In [11]:
advanced_df_raw.isna().sum()

Rk                   0
Player               0
Pos                  0
Age                  0
Tm                   0
G                    0
MP                   0
PER                  0
TS%                  3
3PAr                 3
FTr                  3
ORB%                 0
DRB%                 0
TRB%                 0
AST%                 0
STL%                 0
BLK%                 0
TOV%                 1
USG%                 0
OWS                  0
DWS                  0
WS                   0
WS_48                0
OBPM                 0
DBPM                 0
BPM                  0
VORP                 0
Player_additional    0
dtype: int64

In [42]:
advanced_df_raw[advanced_df_raw['TOV%'].isna()]

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
196,151,Michael Foster Jr.,PF,20,PHI,1,1,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.01,-7.2,-1.9,-9.2,0.0,fostemi02


In [12]:
advanced_df_raw[advanced_df_raw['TS%'].isna()]

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
89,66,Moses Brown,C,23,BRK,2,6,-2.6,NaN,NaN,...,7.4,0.0,0.0,0.0,-0.129,-12.7,2.8,-9.9,0.0,brownmo01
196,151,Michael Foster Jr.,PF,20,PHI,1,1,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.010,-7.2,-1.9,-9.2,0.0,fostemi02
652,515,Alondes Williams,SG,23,BRK,1,5,-20.9,NaN,NaN,...,17.7,-0.1,0.0,-0.1,-0.517,-21.3,-5.2,-26.5,0.0,willial06


In [13]:
advanced_df_raw[advanced_df_raw['3PAr'].isna()]

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
89,66,Moses Brown,C,23,BRK,2,6,-2.6,NaN,NaN,...,7.4,0.0,0.0,0.0,-0.129,-12.7,2.8,-9.9,0.0,brownmo01
196,151,Michael Foster Jr.,PF,20,PHI,1,1,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.010,-7.2,-1.9,-9.2,0.0,fostemi02
652,515,Alondes Williams,SG,23,BRK,1,5,-20.9,NaN,NaN,...,17.7,-0.1,0.0,-0.1,-0.517,-21.3,-5.2,-26.5,0.0,willial06


In [14]:
advanced_df_raw[advanced_df_raw['FTr'].isna()]

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
89,66,Moses Brown,C,23,BRK,2,6,-2.6,NaN,NaN,...,7.4,0.0,0.0,0.0,-0.129,-12.7,2.8,-9.9,0.0,brownmo01
196,151,Michael Foster Jr.,PF,20,PHI,1,1,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.010,-7.2,-1.9,-9.2,0.0,fostemi02
652,515,Alondes Williams,SG,23,BRK,1,5,-20.9,NaN,NaN,...,17.7,-0.1,0.0,-0.1,-0.517,-21.3,-5.2,-26.5,0.0,willial06


### 3.1.5. <a id='toc3_1_5_'></a>[Do these players have multiple lines due to team exchanges?](#toc0_)
- Moses Brown do appear in three different rows once he was traded two times during this season so it may be a good alternative to join the rows
- Michael Foster Jr. and Alondes Williams don't appear. So the missing data may be due to impossobilities to calculate it. It may be a good solution to use 0,0 as values or to attempt to estimate it from the Per Game Dataset.

In [15]:
advanced_df_raw[advanced_df_raw['Player_additional'] == 'brownmo01']

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
87,66,Moses Brown,C,23,TOT,36,294,22.2,0.607,0.0,...,21.2,0.7,0.4,1.1,0.179,0.6,-1.2,-0.6,0.1,brownmo01
88,66,Moses Brown,C,23,LAC,34,288,22.7,0.607,0.0,...,21.5,0.7,0.4,1.1,0.185,0.9,-1.3,-0.4,0.1,brownmo01
89,66,Moses Brown,C,23,BRK,2,6,-2.6,NaN,NaN,...,7.4,0.0,0.0,0.0,-0.129,-12.7,2.8,-9.9,0.0,brownmo01


In [16]:
advanced_df_raw[advanced_df_raw['Player_additional'] == 'fostemi02']

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
196,151,Michael Foster Jr.,PF,20,PHI,1,1,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.01,-7.2,-1.9,-9.2,0.0,fostemi02


In [17]:
advanced_df_raw[advanced_df_raw['Player_additional'] == 'willial06']

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
652,515,Alondes Williams,SG,23,BRK,1,5,-20.9,NaN,NaN,...,17.7,-0.1,0.0,-0.1,-0.517,-21.3,-5.2,-26.5,0.0,willial06


### 3.1.6. <a id='toc3_1_6_'></a>[Let's combine the rows with same players](#toc0_)

In [143]:
advanced_df = advanced_df_raw.groupby("Player_additional", as_index=False).agg(
                      {
                          'Rk':'first', 'Player':'first', 
                          'Pos':'first', 'Age':'first', 
                          'Tm':'first', 'G':'first', 
                          'MP':'mean', 'PER':'mean', 
                          'TS%':'mean', '3PAr':'mean',
                          'FTr':'mean', 'ORB%':'mean', 
                          'DRB%':'mean', 'TRB%':'mean', 
                          'AST%':'mean', 'STL%':'mean', 
                          'BLK%':'mean', 'TOV%':'mean', 
                          'USG%':'mean', 'OWS':'mean', 
                          'DWS':'mean', 'WS':'mean', 
                          'WS_48':'mean', 'OBPM':'mean', 
                          'DBPM':'mean', 'BPM':'mean', 
                          'VORP':'mean', 'Player_additional':'first'
                      }
                      )

#### 3.1.6.1. <a id='toc3_1_6_1_'></a>[Checking if the concatenation went right](#toc0_)

In [144]:
advanced_df.shape[0]

539

In [145]:
advanced_df['Player_additional'].nunique()

539

In [146]:
# Como era:

advanced_df_raw[advanced_df_raw['Player_additional'] == 'brownmo01']

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
87,66,Moses Brown,C,23,TOT,36,294,22.2,0.607,0.0,0.75,22.0,30.9,26.5,2.1,0.7,4.2,10.5,21.2,0.7,0.4,1.1,0.179,0.6,-1.2,-0.6,0.1,brownmo01
88,66,Moses Brown,C,23,LAC,34,288,22.7,0.607,0.0,0.75,22.4,31.5,27.0,2.2,0.5,4.3,9.9,21.5,0.7,0.4,1.1,0.185,0.9,-1.3,-0.4,0.1,brownmo01
89,66,Moses Brown,C,23,BRK,2,6,-2.6,NaN,NaN,NaN,0.0,0.0,0.0,0.0,8.1,0.0,100.0,7.4,0.0,0.0,0.0,-0.129,-12.7,2.8,-9.9,0.0,brownmo01


In [147]:
# Como ficou:

advanced_df[advanced_df['Player_additional'] == 'brownmo01']

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
65,66,Moses Brown,C,23,TOT,36,196.0,14.1,0.607,0.0,0.75,14.8,20.8,17.833333,1.433333,3.1,2.833333,40.133333,16.7,0.466667,0.266667,0.733333,0.078333,-3.733333,0.1,-3.633333,0.066667,brownmo01


### 3.1.7. <a id='toc3_1_7_'></a>[First glance at the Advanced Dataset](#toc0_)

In [148]:
# The data types are all set correctly

advanced_df.dtypes

Rk                     int64
Player                object
Pos                   object
Age                    int64
Tm                    object
G                      int64
MP                   float64
PER                  float64
TS%                  float64
3PAr                 float64
FTr                  float64
ORB%                 float64
DRB%                 float64
TRB%                 float64
AST%                 float64
STL%                 float64
BLK%                 float64
TOV%                 float64
USG%                 float64
OWS                  float64
DWS                  float64
WS                   float64
WS_48                float64
OBPM                 float64
DBPM                 float64
BPM                  float64
VORP                 float64
Player_additional     object
dtype: object

In [149]:
advanced_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Rk,539.0,270.000000,155.740168,1.000,135.500000,270.000,404.500000,539.000
Age,539.0,25.727273,4.290326,19.000,23.000000,25.000,28.500000,42.000
G,539.0,48.042672,24.648006,1.000,30.500000,54.000,68.000000,83.000
MP,539.0,1058.086580,815.685148,1.000,273.500000,914.000,1772.500000,2842.000
PER,539.0,13.318429,6.117492,-20.900,10.100000,13.000,16.400000,65.600
TS%,537.0,0.562687,0.098285,0.000,0.524000,0.567,0.610000,1.064
3PAr,537.0,0.401669,0.219115,0.000,0.261000,0.408,0.547000,1.000
FTr,537.0,0.250840,0.182275,0.000,0.143000,0.230,0.321667,2.000
ORB%,539.0,5.213358,4.241112,0.000,2.166667,3.900,7.050000,28.800
DRB%,539.0,14.951515,6.539374,0.000,10.750000,13.400,18.700000,55.400


In [150]:
advanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rk                 539 non-null    int64  
 1   Player             539 non-null    object 
 2   Pos                539 non-null    object 
 3   Age                539 non-null    int64  
 4   Tm                 539 non-null    object 
 5   G                  539 non-null    int64  
 6   MP                 539 non-null    float64
 7   PER                539 non-null    float64
 8   TS%                537 non-null    float64
 9   3PAr               537 non-null    float64
 10  FTr                537 non-null    float64
 11  ORB%               539 non-null    float64
 12  DRB%               539 non-null    float64
 13  TRB%               539 non-null    float64
 14  AST%               539 non-null    float64
 15  STL%               539 non-null    float64
 16  BLK%               539 non

In [151]:
# Generate a dataset profile report

# advanced_profile = ProfileReport(advanced_df, title = 'Advanced NBA Dataset Profile')
# advanced_profile.to_file('advanced_profile.html')
# advanced_profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### 3.1.8. <a id='toc3_1_8_'></a>[Imputing values to the missing data](#toc0_)
- We still have two players with missing values:
  - Michael Foster Jr.: 'TS%', '3PAr', 'FTr' and 'TOV%'
  - Alondes Williams: 'TS%', '3PAr' and 'FTr'
- Both of them are note playing in NBA league currently
- For that reason we will imput zeros to the NAs

In [112]:
advanced_df[(advanced_df['Player_additional']=='fostemi02') | (advanced_df['Player_additional']=='willial06')]

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,Player_additional
150,151,Michael Foster Jr.,PF,20,PHI,1,1,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.010,-7.2,-1.9,-9.2,0.0,fostemi02
514,515,Alondes Williams,SG,23,BRK,1,5,-20.9,NaN,NaN,NaN,0.0,22.0,11.2,0.0,0.0,0.0,100.0,17.7,-0.1,0.0,-0.1,-0.517,-21.3,-5.2,-26.5,0.0,willial06


In [113]:
advanced_df = advanced_df.fillna(0)

In [114]:
# Checking if the imputation gone well

advanced_df[(advanced_df['Player_additional']=='fostemi02') | (advanced_df['Player_additional']=='willial06')][['Player', 'TS%', '3PAr', 'FTr', 'TOV%']]

,Player,TS%,3PAr,FTr,TOV%
150,Michael Foster Jr.,0.0,0.0,0.0,0.0
514,Alondes Williams,0.0,0.0,0.0,100.0


## 3.2. <a id='toc3_2_'></a>[Examining **Per Game Dataset**](#toc0_)

### 3.2.1. <a id='toc3_2_1_'></a>[Features from Per Game Dataset](#toc0_)


- Rk -- Rank

- Pos -- Position

- Age -- Player's age on February 1 of the season

- Tm -- Team

- G -- Games

- GS -- Games Started

- MP -- Minutes Played Per Game

- FG -- Field Goals Per Game

- FGA -- Field Goal Attempts Per Game

- FG% -- Field Goal Percentage

- 3P -- 3-Point Field Goals Per Game

- 3PA -- 3-Point Field Goal Attempts Per Game

- 3P% -- 3-Point Field Goal Percentage

- 2P -- 2-Point Field Goals Per Game

- 2PA -- 2-Point Field Goal Attempts Per Game

- 2P% -- 2-Point Field Goal Percentage

- eFG% -- Effective Field Goal Percentage

- This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.

- FT -- Free Throws Per Game

- FTA -- Free Throw Attempts Per Game

- FT% -- Free Throw Percentage

- ORB -- Offensive Rebounds Per Game

- DRB -- Defensive Rebounds Per Game

- TRB -- Total Rebounds Per Game

- AST -- Assists Per Game

- STL -- Steals Per Game

- BLK -- Blocks Per Game

- TOV -- Turnovers Per Game

- PF -- Personal Fouls Per Game

- PTS -- Points Per Game

### 3.2.2. <a id='toc3_2_2_'></a>[What are we dealing with?](#toc0_)

In [115]:
pergame_df_raw.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player_additional
0,1,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,0.485,0.5,2.0,0.269,3.0,5.4,0.564,0.521,1.6,2.3,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2,achiupr01
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,0.0,0.0,0.000,3.7,6.2,0.599,0.597,1.1,3.1,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6,adamsst01
2,3,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,0.540,0.0,0.2,0.083,8.0,14.7,0.545,0.541,4.3,5.4,0.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4,adebaba01
3,4,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,0.427,1.4,3.9,0.355,1.4,2.7,0.532,0.532,0.9,1.2,0.812,0.7,1.3,2.1,1.1,0.3,0.3,0.7,1.7,7.9,agbajoc01
4,5,Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,0.470,1.2,3.5,0.353,2.0,3.4,0.591,0.560,1.4,1.9,0.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0,aldamsa01


In [116]:
pergame_df_raw.shape

(679, 31)

### 3.2.3. <a id='toc3_2_3_'></a>[Renaming the columns](#toc0_)

In [117]:
pergame_df_raw.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Player_additional'],
      dtype='object')

In [118]:
pergame_df_raw.columns = ['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Player_additional']

### 3.2.4. <a id='toc3_2_4_'></a>[Checking for NAs](#toc0_)
- In this dataset we have a little bit more NAs than in the previous one
- There are NAs in five columns in total:
  - FG%
  - 3P%
  - 2P% 
  - eFG%
  - FT%
- To the features 'FG%' and 'eFG%' the same thre player from the previous dataset have missing values and we can proceed as we did then

In [119]:
pergame_df_raw.isna().sum()

Rk                    0
Player                0
Pos                   0
Age                   0
Tm                    0
G                     0
GS                    0
MP                    0
FG                    0
FGA                   0
FG%                   3
3P                    0
3PA                   0
3P%                  24
2P                    0
2PA                   0
2P%                   7
eFG%                  3
FT                    0
FTA                   0
FT%                  37
ORB                   0
DRB                   0
TRB                   0
AST                   0
STL                   0
BLK                   0
TOV                   0
PF                    0
PTS                   0
Player_additional     0
dtype: int64

In [120]:
pergame_df_raw[pergame_df_raw['FG%'].isna()]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player_additional
89,66,Moses Brown,C,23,BRK,2,0,3.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.5,0.0,brownmo01
196,151,Michael Foster Jr.,PF,20,PHI,1,0,1.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,fostemi02
652,515,Alondes Williams,SG,23,BRK,1,0,5.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,willial06


In [121]:
pergame_df_raw[pergame_df_raw['3P%'].isna()]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player_additional
21,18,Udoka Azubuike,C,23,UTA,36,4,10.0,1.6,2.0,0.819,0.0,0.0,NaN,1.6,2.0,0.819,0.819,0.2,0.6,0.350,0.9,2.4,3.3,0.3,0.2,0.4,0.5,0.9,3.5,azubuud01
60,43,Bismack Biyombo,C,30,PHO,61,14,14.3,2.0,3.4,0.578,0.0,0.0,NaN,2.0,3.4,0.578,0.578,0.4,1.1,0.357,1.5,2.8,4.3,0.9,0.3,1.4,0.8,1.9,4.3,biyombi01
87,66,Moses Brown,C,23,TOT,36,1,8.2,1.7,2.7,0.635,0.0,0.0,NaN,1.7,2.7,0.635,0.635,0.9,2.0,0.458,1.6,2.3,3.9,0.1,0.1,0.4,0.4,1.1,4.3,brownmo01
88,66,Moses Brown,C,23,LAC,34,1,8.5,1.8,2.8,0.635,0.0,0.0,NaN,1.8,2.8,0.635,0.635,1.0,2.1,0.458,1.7,2.4,4.1,0.1,0.1,0.4,0.4,1.1,4.6,brownmo01
89,66,Moses Brown,C,23,BRK,2,0,3.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.5,0.0,brownmo01
107,82,Vernon Carey Jr.,C,21,WAS,11,0,2.5,0.2,0.7,0.250,0.0,0.0,NaN,0.2,0.7,0.250,0.250,0.2,0.2,1.000,0.3,0.7,1.0,0.3,0.2,0.2,0.2,0.5,0.5,careyve01
116,88,Justin Champagnie,SF,21,TOR,3,0,3.7,1.0,1.0,1.000,0.0,0.0,NaN,1.0,1.0,1.000,1.000,0.0,0.0,NaN,0.3,1.0,1.3,0.3,0.0,0.0,0.0,0.3,2.0,champju01
127,98,Chance Comanche,C,26,POR,1,0,21.0,3.0,5.0,0.600,0.0,0.0,NaN,3.0,5.0,0.600,0.600,1.0,4.0,0.250,2.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0,7.0,comanch01
196,151,Michael Foster Jr.,PF,20,PHI,1,0,1.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,fostemi02
201,156,Daniel Gafford,C,24,WAS,78,47,20.6,3.7,5.1,0.732,0.0,0.0,NaN,3.7,5.1,0.732,0.732,1.6,2.4,0.679,2.1,3.5,5.6,1.1,0.4,1.3,1.1,2.4,9.0,gaffoda01


In [122]:
pergame_df_raw[pergame_df_raw['2P%'].isna()]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player_additional
72,53,Jamaree Bouyea,PG,23,WAS,1,0,6.0,0.0,1.0,0.000,0.0,1.0,0.000,0.0,0.0,NaN,0.000,0.0,0.0,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,bouyeja01
89,66,Moses Brown,C,23,BRK,2,0,3.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.5,0.0,brownmo01
196,151,Michael Foster Jr.,PF,20,PHI,1,0,1.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,fostemi02
211,166,Jacob Gilyard,PG,24,MEM,1,0,41.0,1.0,3.0,0.333,1.0,3.0,0.333,0.0,0.0,NaN,0.500,0.0,0.0,NaN,0.0,4.0,4.0,7.0,3.0,0.0,2.0,3.0,3.0,gilyaja01
336,263,Trevor Keels,SG,19,NYK,3,0,2.7,0.3,1.3,0.250,0.3,1.3,0.250,0.0,0.0,NaN,0.375,0.0,0.0,NaN,0.0,0.7,0.7,0.0,0.0,0.0,0.0,0.0,1.0,keelstr01
613,482,Stanley Umude,SG,23,DET,1,0,2.0,0.0,1.0,0.000,0.0,1.0,0.000,0.0,0.0,NaN,0.000,2.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,umudest01
652,515,Alondes Williams,SG,23,BRK,1,0,5.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,willial06


In [123]:
pergame_df_raw[pergame_df_raw['eFG%'].isna()]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player_additional
89,66,Moses Brown,C,23,BRK,2,0,3.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.5,0.0,brownmo01
196,151,Michael Foster Jr.,PF,20,PHI,1,0,1.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,fostemi02
652,515,Alondes Williams,SG,23,BRK,1,0,5.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,willial06


In [83]:
pergame_df_raw[pergame_df_raw['FT%'].isna()]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player_additional
16,15,Ryan Arcidiacono,PG,28,TOT,20,4,8.6,0.5,1.9,0.243,0.4,1.2,0.348,0.1,0.7,0.071,0.351,0.0,0.0,NaN,0.0,0.8,0.8,1.2,0.3,0.0,0.4,0.9,1.3,arcidry01
17,15,Ryan Arcidiacono,PG,28,NYK,11,0,2.4,0.1,0.5,0.200,0.1,0.3,0.333,0.0,0.2,0.000,0.300,0.0,0.0,NaN,0.0,0.4,0.4,0.2,0.2,0.0,0.1,0.3,0.3,arcidry01
18,15,Ryan Arcidiacono,PG,28,POR,9,4,16.2,0.9,3.6,0.250,0.8,2.2,0.350,0.1,1.3,0.083,0.359,0.0,0.0,NaN,0.0,1.2,1.2,2.3,0.3,0.0,0.7,1.6,2.6,arcidry01
65,48,Leandro Bolmaro,SG,22,UTA,14,0,4.9,0.2,1.4,0.150,0.0,0.3,0.000,0.2,1.1,0.188,0.150,0.0,0.0,NaN,0.3,0.2,0.5,0.5,0.2,0.1,0.5,0.7,0.4,bolmale01
72,53,Jamaree Bouyea,PG,23,WAS,1,0,6.0,0.0,1.0,0.000,0.0,1.0,0.000,0.0,0.0,NaN,0.000,0.0,0.0,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,bouyeja01
89,66,Moses Brown,C,23,BRK,2,0,3.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.5,0.0,brownmo01
90,67,Sterling Brown,SG,27,LAL,4,0,6.0,0.0,1.0,0.000,0.0,0.5,0.000,0.0,0.5,0.000,0.000,0.0,0.0,NaN,0.8,1.3,2.0,0.5,0.8,0.0,0.0,1.0,0.0,brownst02
98,73,Deonte Burton,SG,29,SAC,2,0,3.0,0.0,1.0,0.000,0.0,0.5,0.000,0.0,0.5,0.000,0.000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,burtode02
99,74,Jared Butler,SG,22,OKC,6,1,12.8,2.5,5.3,0.469,1.2,2.3,0.500,1.3,3.0,0.444,0.578,0.0,0.0,NaN,0.2,0.5,0.7,1.3,0.8,0.0,0.8,0.8,6.2,butleja02
113,87,Julian Champagnie,SF,21,PHI,2,0,3.5,0.0,1.0,0.000,0.0,0.5,0.000,0.0,0.5,0.000,0.000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,champju02


### 3.2.5. <a id='toc3_2_5_'></a>[Let's combine multiple player rows in one](#toc0_)

In [134]:
pergame_df = pergame_df_raw.groupby("Player_additional", as_index=False).agg(
                      {
                          'Rk':'first', 'Player':'first', 
                          'Pos':'first', 'Age':'first', 
                          'Tm':'first', 'G':'first', 
                          'GS':'first', 'MP':'mean', 
                          'FG':'mean', 'FGA':'mean', 
                          'FG%':'mean', '3P':'mean', 
                          '3PA':'mean', '3P%':'mean', 
                          '2P':'mean', '2PA':'mean', 
                          '2P%':'mean', 'eFG%':'mean', 
                          'FT':'mean', 'FTA':'mean', 
                          'FT%':'mean', 'ORB':'mean', 
                          'DRB':'mean', 'TRB':'mean', 
                          'AST':'mean', 'STL':'mean', 
                          'BLK':'mean', 'TOV':'mean', 
                          'PF':'mean', 'PTS':'mean', 
                          'Player_additional':'first'
                      }
                      )

#### 3.2.5.1. <a id='toc3_2_5_1_'></a>[Checking if the concatanation went as expected](#toc0_)

In [135]:
print(pergame_df.shape[0], 'out of', pergame_df_raw.shape[0])

539 out of 679


In [136]:
pergame_df['Player_additional'].nunique()

539

#### 3.2.5.2. <a id='toc3_2_5_2_'></a>[Checking again for NAs](#toc0_)
- We still have some NAs. Letś examine them further and decide how to deal with them

In [137]:
pergame_df.isna().sum()

Rk                    0
Player                0
Pos                   0
Age                   0
Tm                    0
G                     0
GS                    0
MP                    0
FG                    0
FGA                   0
FG%                   2
3P                    0
3PA                   0
3P%                  16
2P                    0
2PA                   0
2P%                   5
eFG%                  2
FT                    0
FTA                   0
FT%                  24
ORB                   0
DRB                   0
TRB                   0
AST                   0
STL                   0
BLK                   0
TOV                   0
PF                    0
PTS                   0
Player_additional     0
dtype: int64

### 3.2.6. <a id='toc3_2_6_'></a>[Filling out NAs](#toc0_)
- The NAs still present in the dataset are due to a basic game statistic that has itself only null values (zeros)
- Because of that we can input zeros to the NAs

In [138]:
pergame_df = pergame_df.fillna(0)

In [139]:
pergame_df.isna().sum()

Rk                   0
Player               0
Pos                  0
Age                  0
Tm                   0
G                    0
GS                   0
MP                   0
FG                   0
FGA                  0
FG%                  0
3P                   0
3PA                  0
3P%                  0
2P                   0
2PA                  0
2P%                  0
eFG%                 0
FT                   0
FTA                  0
FT%                  0
ORB                  0
DRB                  0
TRB                  0
AST                  0
STL                  0
BLK                  0
TOV                  0
PF                   0
PTS                  0
Player_additional    0
dtype: int64

### 3.2.7. <a id='toc3_2_7_'></a>[First glance at the Per Game Dataset](#toc0_)

In [140]:
pergame_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Rk,539.0,270.000000,155.740168,1.0,135.500000,270.000000,404.500000,539.0
Age,539.0,25.727273,4.290326,19.0,23.000000,25.000000,28.500000,42.0
G,539.0,48.042672,24.648006,1.0,30.500000,54.000000,68.000000,83.0
GS,539.0,22.820037,27.295285,0.0,1.000000,8.000000,46.500000,83.0
MP,539.0,19.752072,9.563387,1.0,12.266667,19.266667,28.300000,41.0
FG,539.0,3.341868,2.437476,0.0,1.600000,2.600000,4.483333,11.2
FGA,539.0,7.089858,4.958450,0.0,3.400000,5.800000,9.400000,22.2
FG%,539.0,0.463769,0.110612,0.0,0.415500,0.456000,0.508000,1.0
3P,539.0,0.988745,0.873156,0.0,0.300000,0.800000,1.500000,4.9
3PA,539.0,2.778108,2.243782,0.0,1.000000,2.400000,4.100000,11.4


In [141]:
pergame_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rk                 539 non-null    int64  
 1   Player             539 non-null    object 
 2   Pos                539 non-null    object 
 3   Age                539 non-null    int64  
 4   Tm                 539 non-null    object 
 5   G                  539 non-null    int64  
 6   GS                 539 non-null    int64  
 7   MP                 539 non-null    float64
 8   FG                 539 non-null    float64
 9   FGA                539 non-null    float64
 10  FG%                539 non-null    float64
 11  3P                 539 non-null    float64
 12  3PA                539 non-null    float64
 13  3P%                539 non-null    float64
 14  2P                 539 non-null    float64
 15  2PA                539 non-null    float64
 16  2P%                539 non

In [142]:
# Generate a dataset profile report

# pergame_profile = ProfileReport(pergame_df, title = 'Per Game NBA Dataset Profile')
# pergame_profile.to_file('pergame_profile.html')
# pergame_profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# 4. <a id='toc4_'></a>[Feature Engineering and Hypothesis Creation](#toc0_)
- Mental hypothesis map
- Hypothesis list
- Fillout NAs
- Derive new variables as needed

# 5. <a id='toc5_'></a>[Data selection and filtering](#toc0_)
- Filter data rows
- Filter data columns

# 6. <a id='toc6_'></a>[Exploratory Data Analysis](#toc0_)
- Answer the hypothesis list
- Build data visualization solutions and plots

# 7. <a id='toc7_'></a>[Data Preparation](#toc0_)
- Normalize, re-scale and transform (enconding) variables to suit model requirements

# 8. <a id='toc8_'></a>[Feature Selection through Boruta algorithm](#toc0_)
- Use Boruta algorithm to select best features to machine learning models

# 9. <a id='toc9_'></a>[Model implementation](#toc0_)
- Implement different machine learning models and algorithms
- Conduct cross-velidation computing
- Conduct single performance metrics computing

# 10. <a id='toc10_'></a>[Hyperparameter Fine-Tuning](#toc0_)
- Implement hyperparameter search (Bayes Search) to find best model hyperparameter values
- Re-train model using best values

# 11. <a id='toc11_'></a>[Model Error Estimation and Interpretation](#toc0_)
- Use model errors to interpret the goals 

# 12. <a id='toc12_'></a>[Model Deployment](#toc0_)
- Deploy the model to a cloud service so it can be used by its consumers